In [ ]:
import numpy as numpy
import pandas as pd
import ast

import nltk
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
mv=pd.read_csv('/content/drive/MyDrive/tmdb_5000_movies.csv')
cd=pd.read_csv('/content/drive/MyDrive/tmdb_5000_credits.csv')

In [ ]:
mv=mv.merge(cd,on='title')

In [ ]:
mv.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
mv.original_language.value_counts()

en    4510
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
ko      12
cn      12
ru      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: original_language, dtype: int64

In [ ]:
mv=mv.drop(['id','budget','homepage','original_language','original_title','popularity','production_companies','production_countries','release_date','revenue','runtime','spoken_languages','status','tagline','vote_average','vote_count'],axis=1)

In [ ]:
mv.head(1)

,genres,keywords,overview,title,movie_id,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
mv=mv[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
mv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.6+ KB


In [ ]:
mv.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [ ]:
mv.dropna(inplace=True)

In [ ]:
mv.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [ ]:
mv.duplicated().sum()

0

In [ ]:
mv.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [ ]:
def convert(field):
  l=[]
  for i in ast.literal_eval(field):
    l.append(i['name'])
  return l

In [ ]:
mv['genres']=mv['genres'].apply(convert)

In [ ]:
mv.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
mv['keywords']=mv['keywords'].apply(convert)

In [ ]:
mv.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
#extracting only the first three cast from the dictionaries of cast

def convert_cast(field):
  l=[]
  counter=0
  for i in ast.literal_eval(field):
    if counter !=3:
      l.append(i['name'])
      counter+=1
    else:
      break
  return l

In [ ]:
mv['cast']=mv['cast'].apply(convert_cast)

In [ ]:
mv.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
#extracting only the necessary crew 
def fetch_crew(field):
  l=[]
  for i in ast.literal_eval(field):
    if i['job'] == 'Director':
      l.append(i['name'])
      break
  return l

In [ ]:
mv['crew']=mv['crew'].apply(fetch_crew)

In [ ]:
mv.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [ ]:
mv['overview']=mv['overview'].apply(lambda x:x.split())

In [ ]:
mv.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [ ]:
#removing the spaces between words
mv['genres']=mv['genres'].apply(lambda x:[i.replace(" ","") for i in x])
mv['crew']=mv['crew'].apply(lambda x:[i.replace(" ","") for i in x])
mv['keywords']=mv['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
mv['cast']=mv['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
mv.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [ ]:
mv['tags'] = mv['overview']+mv['genres']+mv['keywords']+mv['cast']+mv['crew']

In [ ]:
df=mv[['movie_id','title','tags']]

In [ ]:
df['tags']=df['tags'].apply(lambda x:" ".join(x))
df['tags']=df['tags'].apply(lambda x:x.lower())

In [ ]:
df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [ ]:
#Vecotrizing tags
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vec=cv.fit_transform(df['tags']).toarray()

In [ ]:
len(vec[0])

5000

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [ ]:
df['tags']=df['tags'].apply(stem)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#finding cosine similarity

sim=cosine_similarity(vec)

In [ ]:
sim[0]

array([1.        , 0.08964215, 0.05976143, ..., 0.02519763, 0.02817181,
       0.        ])

In [ ]:
def recommend(movie):
  index=df[df['title'] == movie].index[0]
  dist = sim[index]
  mov = sorted(list(enumerate(dist)),reverse=True,key=lambda x:x[1])[1:6]
  for i in mov:AA
    print(df.iloc[i[0]].title)
    

In [ ]:
recommend('Superman')

Superman Returns
Superman II
Iron Man 2
Superman III
Superman IV: The Quest for Peace


In [ ]:
pickle.dump(df.to_dict(),open('/content/drive/MyDrive/dict_movies.pkl','wb'))

In [ ]:
pickle.dump(sim,open('/content/drive/MyDrive/similar.pkl','wb'))